<a href="https://colab.research.google.com/github/KenKomeji/PredictHomePrice/blob/main/signatePredictPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn.preprocessing as sp
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 
from sklearn import tree
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn import metrics

from sklearn.model_selection import GridSearchCV

import functools

from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_squared_error #MSE
from sklearn.metrics import make_scorer

In [55]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 200

In [56]:
train_org=pd.read_csv('/content/drive/MyDrive/predictprice/train.csv')
test_org=pd.read_csv('/content/drive/MyDrive/predictprice/test.csv')
# sample=pd.read_csv('/content/drive/MyDrive/sample_submit.csv')

In [57]:
train_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 47 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              3500 non-null   int64 
 1   Order           3500 non-null   int64 
 2   MS SubClass     3500 non-null   int64 
 3   MS Zoning       3500 non-null   object
 4   Lot Area        3500 non-null   int64 
 5   Lot Shape       3500 non-null   object
 6   Land Contour    3500 non-null   object
 7   Lot Config      3500 non-null   object
 8   Neighborhood    3500 non-null   object
 9   Bldg Type       3500 non-null   object
 10  House Style     3500 non-null   object
 11  Overall Qual    3500 non-null   int64 
 12  Overall Cond    3500 non-null   int64 
 13  Year Built      3500 non-null   int64 
 14  Year Remod/Add  3500 non-null   int64 
 15  Roof Style      3500 non-null   object
 16  Exterior 1st    3500 non-null   object
 17  Exterior 2nd    3500 non-null   object
 18  Exter Qu

In [58]:
train_org.head()

,id,Order,MS SubClass,MS Zoning,Lot Area,Lot Shape,Land Contour,Lot Config,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Exterior 1st,Exterior 2nd,Exter Qual,Foundation,BsmtFin SF 1,Bsmt Unf SF,Total Bsmt SF,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Gr Liv Area,Bsmt Full Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Fireplaces,Garage Cars,Garage Area,Paved Drive,Wood Deck SF,Open Porch SF,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,0,1611,53,RL,11230,Reg,Lvl,Inside,NoRidge,1Fam,2Story,6,5,1984,2005,Hip,HdBoard,MetalSd,Gd,PConc,0,1260,1151,Ex,Y,SBrkr,1192,1611,2635,0,1,0,3,0,Gd,7,0,2,556,Y,57,0,2,2008,WD,Normal,259175
1,1,2078,56,RL,7855,IR1,Lvl,Inside,Gilbert,1Fam,2Story,6,5,2010,2008,Gable,VinylSd,VinylSd,Gd,PConc,0,967,1035,Ex,Y,SBrkr,892,830,1852,0,2,0,2,0,Gd,6,0,1,563,Y,31,51,1,2007,New,Partial,220872
2,2,2112,17,RL,6178,Reg,Lvl,Inside,Somerst,1Fam,1Story,5,4,2011,2009,Gable,VinylSd,VinylSd,TA,PConc,0,914,890,Gd,Y,SBrkr,1220,0,1449,0,2,0,2,0,Gd,6,0,1,566,Y,0,0,4,2007,WD,Normal,174084
3,3,755,17,RL,9114,Reg,Lvl,Inside,NAmes,1Fam,1Story,3,5,1958,1972,Gable,VinylSd,VinylSd,Gd,PConc,271,89,1864,Ex,Y,SBrkr,1531,0,1833,0,2,0,2,0,TA,6,0,2,1107,Y,33,14,7,2009,WD,Normal,191610
4,4,1601,18,RL,7821,Reg,Lvl,Inside,NAmes,1Fam,1Story,7,6,1966,1971,Gable,MetalSd,MetalSd,Gd,CBlock,0,765,1281,Ex,Y,SBrkr,1393,0,1384,0,1,0,2,0,TA,5,0,0,378,Y,0,0,9,2008,WD,Normal,208007


In [59]:
#以下前処理

# 前処理

In [60]:
# Y/N ~ Central Air
#Great or Poor ~ Exter Qual, Heating QC, Kitchen Qual,Lot Shape
#category ~ MS Zoning, Land Contour??, Neighborhood, Bldg Type, House Style, Roof Style, Foundation, Electrical, Sale Type, Sale Condition,Lot Config, Paved Drive
#special ~ Exterior1st&2nd

In [61]:
#copy_org
train_prepro = train_org.copy()
test_prepro = test_org.copy()

In [62]:
#Great or Poor
#Exter Qual
def GPprepro(data):
    dict1 ={'Ex':5, 'Gd':4, 'TA':3, 'Fa':2,'Po':1}
    data['Exter Qual'] = data['Exter Qual'].map(dict1)
    #Heating QC
    data['Heating QC'] = data['Heating QC'].map(dict1)
    #Kitchen Qual
    data['Kitchen Qual'] = data['Kitchen Qual'].map(dict1)
    #Lot Shape
    dict2 = {'Reg':4, 'IR1':3, 'IR2': 2, 'IR3': 1}
    data['Lot Shape'] = data['Lot Shape'].map(dict2)

GPprepro(train_prepro)
GPprepro(test_prepro)

In [63]:
def categorization(data1,data2,column_name):
  categories = set(data1[column_name].unique().tolist() + data2[column_name].unique().tolist())
  print(column_name,categories,sep='\t')

  data1[column_name] = pd.Categorical(data1[column_name], categories=categories)
  data2[column_name] = pd.Categorical(data2[column_name], categories=categories)

In [64]:
#categorization
#categorization(train_prepro,test_prepro,'Sale Condition')
category_vars = ('Central Air', 'Paved Drive','MS Zoning', 'Land Contour','Neighborhood','Bldg Type','House Style', 'Roof Style', 'Foundation', 'Electrical', 'Sale Type','Sale Condition','Lot Config')
for column_name in category_vars:
  categorization(data1=train_prepro,data2=test_prepro,column_name=column_name)
#map(lambda x: categorization(train_prepro,test_prepro,x),category_var)

Central Air	{'N', 'Y'}
Paved Drive	{'P', 'N', 'Y'}
MS Zoning	{'A (agr)', 'RL', 'FV', 'RM', 'RH'}
Land Contour	{'Low', 'Bnk', 'HLS', 'Lvl'}
Neighborhood	{'IDOTRR', 'Timber', 'Mitchel', 'NoRidge', 'NPkVill', 'SawyerW', 'StoneBr', 'NridgHt', 'Blueste', 'MeadowV', 'SWISU', 'ClearCr', 'BrDale', 'Edwards', 'CollgCr', 'Blmngtn', 'NAmes', 'Crawfor', 'Sawyer', 'NWAmes', 'BrkSide', 'Somerst', 'Veenker', 'OldTown', 'Gilbert'}
Bldg Type	{'2fmCon', 'Duplex', 'Twnhs', '1Fam', 'TwnhsE'}
House Style	{'2.5Unf', '2Story', '1.5Fin', '1Story', 'SLvl', 'SFoyer'}
Roof Style	{'Hip', 'Gable'}
Foundation	{'PConc', 'CBlock', 'Slab', 'BrkTil'}
Electrical	{'FuseA', 'SBrkr', 'FuseF'}
Sale Type	{'New', 'ConLD', 'CWD', 'WD ', 'COD', 'ConLI'}
Sale Condition	{'Abnorml', 'Partial', 'Normal'}
Lot Config	{'Corner', 'FR2', 'Inside', 'CulDSac'}


In [65]:
#drop
drop_col = ['Exterior 1st','Exterior 2nd']
train_prepro.drop(drop_col,axis=1, inplace=True)
test_prepro.drop(drop_col,axis=1, inplace=True)

In [66]:
train_prepro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 45 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   id              3500 non-null   int64   
 1   Order           3500 non-null   int64   
 2   MS SubClass     3500 non-null   int64   
 3   MS Zoning       3500 non-null   category
 4   Lot Area        3500 non-null   int64   
 5   Lot Shape       3500 non-null   int64   
 6   Land Contour    3500 non-null   category
 7   Lot Config      3500 non-null   category
 8   Neighborhood    3500 non-null   category
 9   Bldg Type       3500 non-null   category
 10  House Style     3500 non-null   category
 11  Overall Qual    3500 non-null   int64   
 12  Overall Cond    3500 non-null   int64   
 13  Year Built      3500 non-null   int64   
 14  Year Remod/Add  3500 non-null   int64   
 15  Roof Style      3500 non-null   category
 16  Exter Qual      3500 non-null   int64   
 17  Foundation    

In [67]:
#test_prepro.info()

In [68]:
#get_dummies
train_prepro = pd.get_dummies(train_prepro)
test_prepro = pd.get_dummies(test_prepro)

In [69]:
train_prepro.head()

,id,Order,MS SubClass,Lot Area,Lot Shape,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Exter Qual,BsmtFin SF 1,Bsmt Unf SF,Total Bsmt SF,Heating QC,1st Flr SF,2nd Flr SF,Gr Liv Area,Bsmt Full Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Fireplaces,...,House Style_SLvl,House Style_SFoyer,Roof Style_Hip,Roof Style_Gable,Foundation_PConc,Foundation_CBlock,Foundation_Slab,Foundation_BrkTil,Central Air_N,Central Air_Y,Electrical_FuseA,Electrical_SBrkr,Electrical_FuseF,Paved Drive_P,Paved Drive_N,Paved Drive_Y,Sale Type_New,Sale Type_ConLD,Sale Type_CWD,Sale Type_WD,Sale Type_COD,Sale Type_ConLI,Sale Condition_Abnorml,Sale Condition_Partial,Sale Condition_Normal
0,0,1611,53,11230,4,6,5,1984,2005,4,0,1260,1151,5,1192,1611,2635,0,1,0,3,0,4,7,0,...,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1
1,1,2078,56,7855,3,6,5,2010,2008,4,0,967,1035,5,892,830,1852,0,2,0,2,0,4,6,0,...,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0
2,2,2112,17,6178,4,5,4,2011,2009,3,0,914,890,4,1220,0,1449,0,2,0,2,0,4,6,0,...,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1
3,3,755,17,9114,4,3,5,1958,1972,4,271,89,1864,5,1531,0,1833,0,2,0,2,0,3,6,0,...,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1
4,4,1601,18,7821,4,7,6,1966,1971,4,0,765,1281,5,1393,0,1384,0,1,0,2,0,3,5,0,...,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1


In [70]:
#Central Airの片方はdropさせても問題ない
train_prepro.drop('Central Air_N',axis=1, inplace=True)
test_prepro.drop('Central Air_N',axis=1, inplace=True)

In [71]:
train_prepro.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 103 columns):
 #   Column                  Dtype
---  ------                  -----
 0   id                      int64
 1   Order                   int64
 2   MS SubClass             int64
 3   Lot Area                int64
 4   Lot Shape               int64
 5   Overall Qual            int64
 6   Overall Cond            int64
 7   Year Built              int64
 8   Year Remod/Add          int64
 9   Exter Qual              int64
 10  BsmtFin SF 1            int64
 11  Bsmt Unf SF             int64
 12  Total Bsmt SF           int64
 13  Heating QC              int64
 14  1st Flr SF              int64
 15  2nd Flr SF              int64
 16  Gr Liv Area             int64
 17  Bsmt Full Bath          int64
 18  Full Bath               int64
 19  Half Bath               int64
 20  Bedroom AbvGr           int64
 21  Kitchen AbvGr           int64
 22  Kitchen Qual            int64
 23  TotRms AbvGr

In [72]:
# sns.heatmap(train.corr(),annot=True)

In [73]:
# sns.pairplot(train,hue='disease')

In [74]:
#ss = sp.StandardScaler()
#train_standarzation = ss.fit_transform(X_train)


In [75]:
y_train = train_prepro['SalePrice']
X_train = train_prepro.drop('SalePrice', axis=1)

X_test = test_prepro.copy()

In [76]:
X_train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 102 columns):
 #   Column                  Dtype
---  ------                  -----
 0   id                      int64
 1   Order                   int64
 2   MS SubClass             int64
 3   Lot Area                int64
 4   Lot Shape               int64
 5   Overall Qual            int64
 6   Overall Cond            int64
 7   Year Built              int64
 8   Year Remod/Add          int64
 9   Exter Qual              int64
 10  BsmtFin SF 1            int64
 11  Bsmt Unf SF             int64
 12  Total Bsmt SF           int64
 13  Heating QC              int64
 14  1st Flr SF              int64
 15  2nd Flr SF              int64
 16  Gr Liv Area             int64
 17  Bsmt Full Bath          int64
 18  Full Bath               int64
 19  Half Bath               int64
 20  Bedroom AbvGr           int64
 21  Kitchen AbvGr           int64
 22  Kitchen Qual            int64
 23  TotRms AbvGr

In [77]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=0)  

In [78]:
#stage2

In [79]:
#randomforest

In [80]:
clf = RandomForestRegressor(n_estimators=150, random_state=1) 
clf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=150, random_state=1)

In [81]:
print ('Score: ',clf.score(X_valid, y_valid))

Score:  0.7284332913972293


In [82]:
fti = clf.feature_importances_
print('Feature Importances:')
flist = []
for i, feat in enumerate(X_train.columns.tolist()):
    if(fti[i] > 0.001):
        flist.append(feat)
        print('\t{0:20s} : {1:>.6f}'.format(feat, fti[i]))
flist

Feature Importances:
	id                   : 0.010279
	Order                : 0.011589
	MS SubClass          : 0.008788
	Lot Area             : 0.019030
	Lot Shape            : 0.001494
	Overall Qual         : 0.028565
	Overall Cond         : 0.002768
	Year Built           : 0.047262
	Year Remod/Add       : 0.018968
	Exter Qual           : 0.014080
	BsmtFin SF 1         : 0.025099
	Bsmt Unf SF          : 0.014309
	Total Bsmt SF        : 0.062060
	Heating QC           : 0.003531
	1st Flr SF           : 0.052113
	2nd Flr SF           : 0.005244
	Gr Liv Area          : 0.080080
	Full Bath            : 0.001786
	Bedroom AbvGr        : 0.004753
	Kitchen Qual         : 0.129433
	TotRms AbvGrd        : 0.007885
	Fireplaces           : 0.001386
	Garage Cars          : 0.326079
	Garage Area          : 0.063116
	Wood Deck SF         : 0.010370
	Open Porch SF        : 0.007470
	Mo Sold              : 0.006512
	Yr Sold              : 0.002584
	Lot Config_Corner    : 0.001013
	Neighborhood_NridgHt 

['id',
 'Order',
 'MS SubClass',
 'Lot Area',
 'Lot Shape',
 'Overall Qual',
 'Overall Cond',
 'Year Built',
 'Year Remod/Add',
 'Exter Qual',
 'BsmtFin SF 1',
 'Bsmt Unf SF',
 'Total Bsmt SF',
 'Heating QC',
 '1st Flr SF',
 '2nd Flr SF',
 'Gr Liv Area',
 'Full Bath',
 'Bedroom AbvGr',
 'Kitchen Qual',
 'TotRms AbvGrd',
 'Fireplaces',
 'Garage Cars',
 'Garage Area',
 'Wood Deck SF',
 'Open Porch SF',
 'Mo Sold',
 'Yr Sold',
 'Lot Config_Corner',
 'Neighborhood_NridgHt',
 'Neighborhood_CollgCr',
 'Neighborhood_NAmes',
 'House Style_2Story',
 'Foundation_PConc',
 'Foundation_CBlock',
 'Foundation_BrkTil']

In [83]:
y_train = train_prepro['SalePrice']
X_train = train_prepro.drop('SalePrice', axis=1)
X_train = X_train[flist]
X_train = X_train.drop('id',axis=1)

X_test = test_prepro.copy()
X_test = X_test[flist]
X_test = X_test.drop('id',axis=1)



In [84]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=0)  

In [85]:
#stratifiedkfold = StratifiedKFold(n_splits=5)  #K=10分割
#scores = cross_val_score(clf, X_train, y_train, cv=stratifiedkfold)
#print('Cross-Validation scores: {}'.format(scores))   # 各分割におけるスコア
#print('Average score: {}'.format(np.mean(scores)))  # スコアの平均値

In [86]:
def rmse(y_true,y_pred):
    #RMSEを算出
    rmse = np.sqrt(mean_squared_error(y_true,y_pred))
    print('rmse',rmse)
    return rmse

In [87]:
'''
param_grid = {"n_estimators":[120,130,150,170,190,210,300,500,1000]}

tree_grid = GridSearchCV(estimator=clf,
                 param_grid = param_grid,     
                 scoring=make_scorer(rmse,greater_is_better=False),  #metrics
                 cv = 5,              #cross-validation
                 n_jobs =-1)          #number of core

tree_grid.fit(X_train,y_train) #fit

tree_grid_best = tree_grid.best_estimator_ #best estimator
print("Best Model Parameter: ",tree_grid.best_params_)
print("Best Model Score    : ",tree_grid.best_score_)
'''

'\nparam_grid = {"n_estimators":[120,130,150,170,190,210,300,500,1000]}\n\ntree_grid = GridSearchCV(estimator=clf,\n                 param_grid = param_grid,     \n                 scoring=make_scorer(rmse,greater_is_better=False),  #metrics\n                 cv = 5,              #cross-validation\n                 n_jobs =-1)          #number of core\n\ntree_grid.fit(X_train,y_train) #fit\n\ntree_grid_best = tree_grid.best_estimator_ #best estimator\nprint("Best Model Parameter: ",tree_grid.best_params_)\nprint("Best Model Score    : ",tree_grid.best_score_)\n'

In [88]:
#xgboost

# xgboost

In [89]:
y_train

177     118610
553     195718
64      123745
725     280045
856     117659
         ...  
835     213588
3264    128981
1653    187720
2607    116230
2732    291170
Name: SalePrice, Length: 3150, dtype: int64

In [90]:
dtrain = xgb.DMatrix(X_train.values, label=y_train)
dvalid = xgb.DMatrix(X_valid.values, label=y_valid)
dtest = xgb.DMatrix(X_test.values)

In [91]:
params = {
        'objective': 'reg:squarederror','silent':1, 'random_state':1234, 
        # 学習用の指標 (RMSE)
        'eval_metric': 'rmse',
        'eta': 0.01,
    }
num_round = 1000
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]#訓練データはdtrain、評価用のテストデータはdvalidと設定

In [92]:
#xgboost_clf = xgb.XGBClassifier()

In [93]:
model = xgb.train(params,
                    dtrain,#訓練データ
                    num_round,#設定した学習回数
                    early_stopping_rounds=30,
                    evals=watchlist,
                    )

[0]	train-rmse:209288	eval-rmse:211169
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 30 rounds.
[1]	train-rmse:207314	eval-rmse:209224
[2]	train-rmse:205359	eval-rmse:207294
[3]	train-rmse:203424	eval-rmse:205407
[4]	train-rmse:201509	eval-rmse:203524
[5]	train-rmse:199612	eval-rmse:201703
[6]	train-rmse:197737	eval-rmse:199865
[7]	train-rmse:195878	eval-rmse:198087
[8]	train-rmse:194040	eval-rmse:196291
[9]	train-rmse:192220	eval-rmse:194551
[10]	train-rmse:190419	eval-rmse:192804
[11]	train-rmse:188636	eval-rmse:191089
[12]	train-rmse:186873	eval-rmse:189360
[13]	train-rmse:185125	eval-rmse:187695
[14]	train-rmse:183398	eval-rmse:186005
[15]	train-rmse:181685	eval-rmse:184365
[16]	train-rmse:179993	eval-rmse:182711
[17]	train-rmse:178315	eval-rmse:181114
[18]	train-rmse:176658	eval-rmse:179487
[19]	train-rmse:175014	eval-rmse:177927
[20]	train-rmse:173390	eval-rmse:176334
[21]	train-rmse:171785	eval

# xgboost GridSearch

In [94]:
from xgboost import XGBRegressor
from sklearn.model_selection import KFold

model = XGBRegressor(booster='gbtree', objective='reg:squarederror',random_state=1234, n_estimators=10000)
cv = KFold(n_splits=3, shuffle=True, random_state=1234)

fit_params = {'verbose': 0,  # 学習中のコマンドライン出力
              'early_stopping_rounds': 30,  # 学習時、評価指標がこの回数連続で改善しなくなった時点でストップ
              'eval_metric': 'rmse',  # early_stopping_roundsの評価指標
              'eval_set': [(X_train, y_train)]  # early_stopping_roundsの評価指標算出用データ
              }

In [95]:
#2021/12/29
'''
cv_params = {'learning_rate': [0.01, 0.3],
             'min_child_weight': [1,2,3],
             'max_depth': [6],
             'colsample_bytree': [0.4,0.5, 0.6],
             'subsample': [0.7, 0.8, 0.9]
             }
最適パラメータ {'colsample_bytree': 0.4, 'learning_rate': 0.01, 'max_depth': 6, 'min_child_weight': 1, 'subsample': 0.9}
スコア -1629243498.1719866
'''

"\ncv_params = {'learning_rate': [0.01, 0.3],\n             'min_child_weight': [1,2,3],\n             'max_depth': [6],\n             'colsample_bytree': [0.4,0.5, 0.6],\n             'subsample': [0.7, 0.8, 0.9]\n             }\n最適パラメータ {'colsample_bytree': 0.4, 'learning_rate': 0.01, 'max_depth': 6, 'min_child_weight': 1, 'subsample': 0.9}\nスコア -1629243498.1719866\n"

In [96]:
cv_params = {'learning_rate': [0.01, 0.03],
             'min_child_weight': [1,1.5],
             'max_depth': [6],
             'colsample_bytree': [0.4,0.45,0.5],
             'subsample': [0.85, 0.9]
             }
# グリッドサーチのインスタンス作成
gridcv = GridSearchCV(model, cv_params, cv=cv,
                      scoring='neg_mean_squared_error', n_jobs=-1)
# グリッドサーチ実行（学習実行）
gridcv.fit(X_train, y_train, **fit_params)
# 最適パラメータの表示と保持
best_params = gridcv.best_params_
best_score = gridcv.best_score_
print(f'最適パラメータ {best_params}\nスコア {best_score}')

最適パラメータ {'colsample_bytree': 0.4, 'learning_rate': 0.01, 'max_depth': 6, 'min_child_weight': 1, 'subsample': 0.9}
スコア -1629243498.1719866


In [46]:
xgbbest = gridcv.best_estimator_

In [48]:
#予測
#prediction_XG = model.predict(dtest, ntree_limit = model.best_ntree_limit)
prediction_XG = xgbbest.predict(X_test)

#小数を丸めている
prediction_XG = np.asarray(prediction_XG,dtype = int)

In [49]:
prediction_XG

array([103504, 127581, 133960, ..., 173097, 254879, 366061])

In [ ]:
X_test.values

array([[  755,    58,  3668, ...,     0,     1,     0],
       [ 1992,    18,  6254, ...,     0,     0,     1],
       [ 2766,    18,   954, ...,     0,     0,     1],
       ...,
       [ 1382,    19, 10167, ...,     0,     0,     1],
       [  887,    55, 10494, ...,     1,     0,     0],
       [ 1008,    18,  7027, ...,     1,     0,     0]])

In [ ]:
#predict

In [50]:
#X_test_sub=test.drop(drop_col, axis=1)
#y_pred = clf.predict(X_sub)
y_pred = prediction_XG

In [51]:
sub=test_org[['id']]
sub=sub.assign(disease=y_pred)
sub.to_csv('/content/drive/MyDrive/submission.csv', header=False,index=False)

In [ ]:
sub.head()

,id,disease
0,3500,102189
1,3501,125288
2,3502,126543
3,3503,139654
4,3504,167507


In [ ]:
fti = tree_grid_best.feature_importances_   

print('Feature Importances:')
for i, feat in enumerate(X_train.columns):
    print('\t{0:20s} : {1:>.6f}'.format(feat, fti[i]))

Feature Importances:
	T_Bil                : 0.217917
	D_Bil                : 0.128594
	ALP                  : 0.144370
	ALT_GPT              : 0.117606
	AST_GOT              : 0.130970
	TP                   : 0.074083
	AG_ratio             : 0.094538
	lr_result            : 0.091922
